<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# Session: Using a Database to Find Solutions for Business Problems 

#### <i>A Microsoft Presentation from the SQL Server team</i>

## Introduction and Setup

<p style="border-bottom: 1px solid lightgrey;"></p>


<h2><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/pin.jpg?raw=true">Finding a solution</h2>

The options available to you to create a soltuion for a given problem can be overwhelming. To begin, you need to understand three general areas:

- The problem
- The options (and components) for solving the problem
- The constraints and variables for the solution/problem instersection

What you're often thinking of is a single problem with many possible components within a solution. Or, the constraints and variables may dictate that you start from the other end - the components - to be either included or excluded from the solution. Once you know the possible options for a solution, you also need to know the details of how the component will be used in the solution. 

Clearly a database can help with defining and setting up these intersections of problem, solution and components. And in particular, a Relational Database, with it's focus on relationships between objects, is ideally suited for the task. This presentation will help you understand how to do that.

<h2><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/textbubble.png?raw=true">Understanding the Solutions Database</h2>

We're using a very simple layout for this database, involving only three primary tables:

- **Problem**: The list of problems in the form of "We want..."
- **Solution**: Solutions composed of various components, processes and optionally services
- **AzureService**: The Azure Services (components) and optionally any services (consulting/staff aug) needed for the solution

There are also two secondary tables to hold information about the solutions and services: 

- **SolutionReferences**: Reference documents, tutorials, and cost estimates for the Solution
- **ServiceReferences**: Reference documents, tutorials, and code samples for the Services


And of course tertiary tables to handle the many-to-many joins:

- **ProblemToSolution**
- **SoltuionToService**

Here is the Entity Relationship Diagram (ERD) for the database:

<p><img style="float: left; margin: 0px 35px 35px 0px;" src="https://github.com/BuckWoody/presentations/blob/master/problemtosolution/graphics/erd.png?raw=true"></p>

You can also edit this simple design to include more elements, such as more detail on the problem, or any other specific information you want.


<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/checkbox.png?raw=true"><b>Activity: Connect to a SQL Server Instance and Create the Database</b></p>

- You'll start by creating a database for the solution mapping. Connect to any currently supported version of SQL Server using the *Attach To...* link at the top of this page, set the Kernel to *SQL*,and either run the script below (which was created from Azure SQL DB) on a Server Instance to Create the Database with no data in it, or
- Use the *DACPAC* file in this directory to restore the database, or
- You can also create the database yourself from the graphic if you like, and make any changes you want.

( [More on using a *DACPAC* here](https://docs.microsoft.com/en-us/sql/relational-databases/data-tier-applications/import-a-bacpac-file-to-create-a-new-user-database?view=sql-server-2017) )


In [1]:
USE master;
GO

/****** Object:  UserDefinedFunction [dbo].[fn_diagramobjects]    Script Date: 6/26/2019 11:14:26 AM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO

	CREATE FUNCTION [dbo].[fn_diagramobjects]() 
	RETURNS int
	WITH EXECUTE AS N'dbo'
	AS
	BEGIN
		declare @id_upgraddiagrams		int
		declare @id_sysdiagrams			int
		declare @id_helpdiagrams		int
		declare @id_helpdiagramdefinition	int
		declare @id_creatediagram	int
		declare @id_renamediagram	int
		declare @id_alterdiagram 	int 
		declare @id_dropdiagram		int
		declare @InstalledObjects	int

		select @InstalledObjects = 0

		select 	@id_upgraddiagrams = object_id(N'dbo.sp_upgraddiagrams'),
			@id_sysdiagrams = object_id(N'dbo.sysdiagrams'),
			@id_helpdiagrams = object_id(N'dbo.sp_helpdiagrams'),
			@id_helpdiagramdefinition = object_id(N'dbo.sp_helpdiagramdefinition'),
			@id_creatediagram = object_id(N'dbo.sp_creatediagram'),
			@id_renamediagram = object_id(N'dbo.sp_renamediagram'),
			@id_alterdiagram = object_id(N'dbo.sp_alterdiagram'), 
			@id_dropdiagram = object_id(N'dbo.sp_dropdiagram')

		if @id_upgraddiagrams is not null
			select @InstalledObjects = @InstalledObjects + 1
		if @id_sysdiagrams is not null
			select @InstalledObjects = @InstalledObjects + 2
		if @id_helpdiagrams is not null
			select @InstalledObjects = @InstalledObjects + 4
		if @id_helpdiagramdefinition is not null
			select @InstalledObjects = @InstalledObjects + 8
		if @id_creatediagram is not null
			select @InstalledObjects = @InstalledObjects + 16
		if @id_renamediagram is not null
			select @InstalledObjects = @InstalledObjects + 32
		if @id_alterdiagram  is not null
			select @InstalledObjects = @InstalledObjects + 64
		if @id_dropdiagram is not null
			select @InstalledObjects = @InstalledObjects + 128
		
		return @InstalledObjects 
	END
	
GO
/****** Object:  Table [dbo].[Problem]    Script Date: 6/26/2019 11:14:26 AM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [dbo].[Problem](
	[ProblemID] [int] IDENTITY(1,1) NOT NULL,
	[ProblemName] [nvarchar](150) NULL,
	[ProblemDescription] [nvarchar](max) NULL,
PRIMARY KEY CLUSTERED 
(
	[ProblemID] ASC
)WITH (STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF) ON [PRIMARY]
) ON [PRIMARY] TEXTIMAGE_ON [PRIMARY]
GO
/****** Object:  Table [dbo].[Solution]    Script Date: 6/26/2019 11:14:26 AM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [dbo].[Solution](
	[SolutionID] [int] IDENTITY(1,1) NOT NULL,
	[SolutionName] [nvarchar](150) NULL,
	[SolutionDescription] [nvarchar](max) NULL,
PRIMARY KEY CLUSTERED 
(
	[SolutionID] ASC
)WITH (STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF) ON [PRIMARY]
) ON [PRIMARY] TEXTIMAGE_ON [PRIMARY]
GO
/****** Object:  Table [dbo].[SolutionReferences]    Script Date: 6/26/2019 11:14:26 AM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [dbo].[SolutionReferences](
	[SolutionReferenceID] [int] IDENTITY(1,1) NOT NULL,
	[SolutionReferenceType] [nvarchar](100) NULL,
	[SolutionReferenceName] [nvarchar](150) NULL,
	[SolutionReferenceDescription] [nvarchar](max) NULL,
	[SolutionID] [int] NULL,
	[SolutionReferencesLocation] [varchar](255) NULL,
PRIMARY KEY CLUSTERED 
(
	[SolutionReferenceID] ASC
)WITH (STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF) ON [PRIMARY]
) ON [PRIMARY] TEXTIMAGE_ON [PRIMARY]
GO
/****** Object:  Table [dbo].[ProblemToSolution]    Script Date: 6/26/2019 11:14:26 AM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [dbo].[ProblemToSolution](
	[ProblemToSolutionID] [int] IDENTITY(1,1) NOT NULL,
	[ProblemID] [int] NULL,
	[SolutionID] [int] NULL,
	[Strength] [nvarchar](50) NULL,
PRIMARY KEY CLUSTERED 
(
	[ProblemToSolutionID] ASC
)WITH (STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF) ON [PRIMARY]
) ON [PRIMARY]
GO
/****** Object:  View [dbo].[ProblemsToSolutions]    Script Date: 6/26/2019 11:14:26 AM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE VIEW [dbo].[ProblemsToSolutions]
AS
SELECT     dbo.Problem.ProblemName, dbo.Problem.ProblemDescription, dbo.Solution.SolutionName, dbo.Solution.SolutionDescription, dbo.ProblemToSolution.Strength, 
                  dbo.SolutionReferences.SolutionReferenceType, dbo.SolutionReferences.SolutionReferenceName, dbo.SolutionReferences.SolutionReferenceDescription, 
                  dbo.SolutionReferences.SolutionReferencesLocation
FROM        dbo.Problem INNER JOIN
                  dbo.ProblemToSolution ON dbo.Problem.ProblemID = dbo.ProblemToSolution.ProblemID INNER JOIN
                  dbo.Solution ON dbo.ProblemToSolution.SolutionID = dbo.Solution.SolutionID INNER JOIN
                  dbo.SolutionReferences ON dbo.Solution.SolutionID = dbo.SolutionReferences.SolutionID
GO
/****** Object:  Table [dbo].[AzureService]    Script Date: 6/26/2019 11:14:26 AM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [dbo].[AzureService](
	[AzureServiceID] [int] IDENTITY(1,1) NOT NULL,
	[ServiceName] [nvarchar](150) NULL,
	[ServiceDescription] [nvarchar](max) NULL,
PRIMARY KEY CLUSTERED 
(
	[AzureServiceID] ASC
)WITH (STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF) ON [PRIMARY]
) ON [PRIMARY] TEXTIMAGE_ON [PRIMARY]
GO
/****** Object:  Table [dbo].[ServiceReferences]    Script Date: 6/26/2019 11:14:26 AM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [dbo].[ServiceReferences](
	[ServiceReferenceID] [int] IDENTITY(1,1) NOT NULL,
	[ServiceReferenceName] [nvarchar](150) NULL,
	[ServiceReferenceType] [nvarchar](50) NULL,
	[ServiceReferenceLocation] [nvarchar](150) NULL,
	[ServiceReferenceDescription] [nvarchar](max) NULL,
	[AzureServiceID] [int] NULL,
PRIMARY KEY CLUSTERED 
(
	[ServiceReferenceID] ASC
)WITH (STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF) ON [PRIMARY]
) ON [PRIMARY] TEXTIMAGE_ON [PRIMARY]
GO
/****** Object:  Table [dbo].[SolutionToService]    Script Date: 6/26/2019 11:14:26 AM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [dbo].[SolutionToService](
	[SolutionToServiceID] [int] IDENTITY(1,1) NOT NULL,
	[SolutionID] [int] NULL,
	[AzureServiceID] [int] NULL,
	[Complexity] [nvarchar](50) NULL,
	[MonthlyEstimatedCost] [nvarchar](50) NULL,
PRIMARY KEY CLUSTERED 
(
	[SolutionToServiceID] ASC
)WITH (STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF) ON [PRIMARY]
) ON [PRIMARY]
GO
/****** Object:  View [dbo].[SolutionsToServices]    Script Date: 6/26/2019 11:14:26 AM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE VIEW [dbo].[SolutionsToServices]
AS
SELECT     dbo.Solution.SolutionName, dbo.AzureService.ServiceName, dbo.AzureService.ServiceDescription, dbo.ServiceReferences.ServiceReferenceName, dbo.ServiceReferences.ServiceReferenceType, 
                  dbo.ServiceReferences.ServiceReferenceLocation
FROM        dbo.Solution INNER JOIN
                  dbo.SolutionToService ON dbo.Solution.SolutionID = dbo.SolutionToService.SolutionID INNER JOIN
                  dbo.AzureService ON dbo.SolutionToService.AzureServiceID = dbo.AzureService.AzureServiceID INNER JOIN
                  dbo.ServiceReferences ON dbo.AzureService.AzureServiceID = dbo.ServiceReferences.AzureServiceID
GO
/****** Object:  Table [dbo].[sysdiagrams]    Script Date: 6/26/2019 11:14:26 AM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [dbo].[sysdiagrams](
	[name] [sysname] NOT NULL,
	[principal_id] [int] NOT NULL,
	[diagram_id] [int] IDENTITY(1,1) NOT NULL,
	[version] [int] NULL,
	[definition] [varbinary](max) NULL,
PRIMARY KEY CLUSTERED 
(
	[diagram_id] ASC
)WITH (STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF) ON [PRIMARY],
 CONSTRAINT [UK_principal_name] UNIQUE NONCLUSTERED 
(
	[principal_id] ASC,
	[name] ASC
)WITH (STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF) ON [PRIMARY]
) ON [PRIMARY] TEXTIMAGE_ON [PRIMARY]
GO
ALTER TABLE [dbo].[ProblemToSolution]  WITH NOCHECK ADD  CONSTRAINT [FK_ProblemToSolution_Problem] FOREIGN KEY([SolutionID])
REFERENCES [dbo].[Problem] ([ProblemID])
GO
ALTER TABLE [dbo].[ProblemToSolution] CHECK CONSTRAINT [FK_ProblemToSolution_Problem]
GO
ALTER TABLE [dbo].[ProblemToSolution]  WITH CHECK ADD  CONSTRAINT [FK_ProblemToSolution_Solution] FOREIGN KEY([ProblemID])
REFERENCES [dbo].[Solution] ([SolutionID])
GO
ALTER TABLE [dbo].[ProblemToSolution] CHECK CONSTRAINT [FK_ProblemToSolution_Solution]
GO
ALTER TABLE [dbo].[ServiceReferences]  WITH CHECK ADD  CONSTRAINT [FK_ServiceReferences_AzureService] FOREIGN KEY([AzureServiceID])
REFERENCES [dbo].[AzureService] ([AzureServiceID])
GO
ALTER TABLE [dbo].[ServiceReferences] CHECK CONSTRAINT [FK_ServiceReferences_AzureService]
GO
ALTER TABLE [dbo].[SolutionReferences]  WITH CHECK ADD  CONSTRAINT [FK_SolutionReferences_Solution1] FOREIGN KEY([SolutionID])
REFERENCES [dbo].[Solution] ([SolutionID])
GO
ALTER TABLE [dbo].[SolutionReferences] CHECK CONSTRAINT [FK_SolutionReferences_Solution1]
GO
ALTER TABLE [dbo].[SolutionToService]  WITH CHECK ADD  CONSTRAINT [FK_SolutionToService_AzureService] FOREIGN KEY([AzureServiceID])
REFERENCES [dbo].[AzureService] ([AzureServiceID])
GO
ALTER TABLE [dbo].[SolutionToService] CHECK CONSTRAINT [FK_SolutionToService_AzureService]
GO
ALTER TABLE [dbo].[SolutionToService]  WITH CHECK ADD  CONSTRAINT [FK_SolutionToService_Solution] FOREIGN KEY([SolutionID])
REFERENCES [dbo].[Solution] ([SolutionID])
GO
ALTER TABLE [dbo].[SolutionToService] CHECK CONSTRAINT [FK_SolutionToService_Solution]
GO
/****** Object:  StoredProcedure [dbo].[sp_alterdiagram]    Script Date: 6/26/2019 11:14:26 AM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO

	CREATE PROCEDURE [dbo].[sp_alterdiagram]
	(
		@diagramname 	sysname,
		@owner_id	int	= null,
		@version 	int,
		@definition 	varbinary(max)
	)
	WITH EXECUTE AS 'dbo'
	AS
	BEGIN
		set nocount on
	
		declare @theId 			int
		declare @retval 		int
		declare @IsDbo 			int
		
		declare @UIDFound 		int
		declare @DiagId			int
		declare @ShouldChangeUID	int
	
		if(@diagramname is null)
		begin
			RAISERROR ('Invalid ARG', 16, 1)
			return -1
		end
	
		execute as caller;
		select @theId = DATABASE_PRINCIPAL_ID();	 
		select @IsDbo = IS_MEMBER(N'db_owner'); 
		if(@owner_id is null)
			select @owner_id = @theId;
		revert;
	
		select @ShouldChangeUID = 0
		select @DiagId = diagram_id, @UIDFound = principal_id from dbo.sysdiagrams where principal_id = @owner_id and name = @diagramname 
		
		if(@DiagId IS NULL or (@IsDbo = 0 and @theId <> @UIDFound))
		begin
			RAISERROR ('Diagram does not exist or you do not have permission.', 16, 1);
			return -3
		end
	
		if(@IsDbo <> 0)
		begin
			if(@UIDFound is null or USER_NAME(@UIDFound) is null) -- invalid principal_id
			begin
				select @ShouldChangeUID = 1 ;
			end
		end

		-- update dds data			
		update dbo.sysdiagrams set definition = @definition where diagram_id = @DiagId ;

		-- change owner
		if(@ShouldChangeUID = 1)
			update dbo.sysdiagrams set principal_id = @theId where diagram_id = @DiagId ;

		-- update dds version
		if(@version is not null)
			update dbo.sysdiagrams set version = @version where diagram_id = @DiagId ;

		return 0
	END
	
GO
/****** Object:  StoredProcedure [dbo].[sp_creatediagram]    Script Date: 6/26/2019 11:14:26 AM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO

	CREATE PROCEDURE [dbo].[sp_creatediagram]
	(
		@diagramname 	sysname,
		@owner_id		int	= null, 	
		@version 		int,
		@definition 	varbinary(max)
	)
	WITH EXECUTE AS 'dbo'
	AS
	BEGIN
		set nocount on
	
		declare @theId int
		declare @retval int
		declare @IsDbo	int
		declare @userName sysname
		if(@version is null or @diagramname is null)
		begin
			RAISERROR (N'E_INVALIDARG', 16, 1);
			return -1
		end
	
		execute as caller;
		select @theId = DATABASE_PRINCIPAL_ID(); 
		select @IsDbo = IS_MEMBER(N'db_owner');
		revert; 
		
		if @owner_id is null
		begin
			select @owner_id = @theId;
		end
		else
		begin
			if @theId <> @owner_id
			begin
				if @IsDbo = 0
				begin
					RAISERROR (N'E_INVALIDARG', 16, 1);
					return -1
				end
				select @theId = @owner_id
			end
		end
		-- next 2 line only for test, will be removed after define name unique
		if EXISTS(select diagram_id from dbo.sysdiagrams where principal_id = @theId and name = @diagramname)
		begin
			RAISERROR ('The name is already used.', 16, 1);
			return -2
		end
	
		insert into dbo.sysdiagrams(name, principal_id , version, definition)
				VALUES(@diagramname, @theId, @version, @definition) ;
		
		select @retval = @@IDENTITY 
		return @retval
	END
	
GO
/****** Object:  StoredProcedure [dbo].[sp_dropdiagram]    Script Date: 6/26/2019 11:14:26 AM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO

	CREATE PROCEDURE [dbo].[sp_dropdiagram]
	(
		@diagramname 	sysname,
		@owner_id	int	= null
	)
	WITH EXECUTE AS 'dbo'
	AS
	BEGIN
		set nocount on
		declare @theId 			int
		declare @IsDbo 			int
		
		declare @UIDFound 		int
		declare @DiagId			int
	
		if(@diagramname is null)
		begin
			RAISERROR ('Invalid value', 16, 1);
			return -1
		end
	
		EXECUTE AS CALLER;
		select @theId = DATABASE_PRINCIPAL_ID();
		select @IsDbo = IS_MEMBER(N'db_owner'); 
		if(@owner_id is null)
			select @owner_id = @theId;
		REVERT; 
		
		select @DiagId = diagram_id, @UIDFound = principal_id from dbo.sysdiagrams where principal_id = @owner_id and name = @diagramname 
		if(@DiagId IS NULL or (@IsDbo = 0 and @UIDFound <> @theId))
		begin
			RAISERROR ('Diagram does not exist or you do not have permission.', 16, 1)
			return -3
		end
	
		delete from dbo.sysdiagrams where diagram_id = @DiagId;
	
		return 0;
	END
	
GO
/****** Object:  StoredProcedure [dbo].[sp_helpdiagramdefinition]    Script Date: 6/26/2019 11:14:26 AM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO

	CREATE PROCEDURE [dbo].[sp_helpdiagramdefinition]
	(
		@diagramname 	sysname,
		@owner_id	int	= null 		
	)
	WITH EXECUTE AS N'dbo'
	AS
	BEGIN
		set nocount on

		declare @theId 		int
		declare @IsDbo 		int
		declare @DiagId		int
		declare @UIDFound	int
	
		if(@diagramname is null)
		begin
			RAISERROR (N'E_INVALIDARG', 16, 1);
			return -1
		end
	
		execute as caller;
		select @theId = DATABASE_PRINCIPAL_ID();
		select @IsDbo = IS_MEMBER(N'db_owner');
		if(@owner_id is null)
			select @owner_id = @theId;
		revert; 
	
		select @DiagId = diagram_id, @UIDFound = principal_id from dbo.sysdiagrams where principal_id = @owner_id and name = @diagramname;
		if(@DiagId IS NULL or (@IsDbo = 0 and @UIDFound <> @theId ))
		begin
			RAISERROR ('Diagram does not exist or you do not have permission.', 16, 1);
			return -3
		end

		select version, definition FROM dbo.sysdiagrams where diagram_id = @DiagId ; 
		return 0
	END
	
GO
/****** Object:  StoredProcedure [dbo].[sp_helpdiagrams]    Script Date: 6/26/2019 11:14:26 AM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO

	CREATE PROCEDURE [dbo].[sp_helpdiagrams]
	(
		@diagramname sysname = NULL,
		@owner_id int = NULL
	)
	WITH EXECUTE AS N'dbo'
	AS
	BEGIN
		DECLARE @user sysname
		DECLARE @dboLogin bit
		EXECUTE AS CALLER;
			SET @user = USER_NAME();
			SET @dboLogin = CONVERT(bit,IS_MEMBER('db_owner'));
		REVERT;
		SELECT
			[Database] = DB_NAME(),
			[Name] = name,
			[ID] = diagram_id,
			[Owner] = USER_NAME(principal_id),
			[OwnerID] = principal_id
		FROM
			sysdiagrams
		WHERE
			(@dboLogin = 1 OR USER_NAME(principal_id) = @user) AND
			(@diagramname IS NULL OR name = @diagramname) AND
			(@owner_id IS NULL OR principal_id = @owner_id)
		ORDER BY
			4, 5, 1
	END
	
GO
/****** Object:  StoredProcedure [dbo].[sp_renamediagram]    Script Date: 6/26/2019 11:14:26 AM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO

	CREATE PROCEDURE [dbo].[sp_renamediagram]
	(
		@diagramname 		sysname,
		@owner_id		int	= null,
		@new_diagramname	sysname
	
	)
	WITH EXECUTE AS 'dbo'
	AS
	BEGIN
		set nocount on
		declare @theId 			int
		declare @IsDbo 			int
		
		declare @UIDFound 		int
		declare @DiagId			int
		declare @DiagIdTarg		int
		declare @u_name			sysname
		if((@diagramname is null) or (@new_diagramname is null))
		begin
			RAISERROR ('Invalid value', 16, 1);
			return -1
		end
	
		EXECUTE AS CALLER;
		select @theId = DATABASE_PRINCIPAL_ID();
		select @IsDbo = IS_MEMBER(N'db_owner'); 
		if(@owner_id is null)
			select @owner_id = @theId;
		REVERT;
	
		select @u_name = USER_NAME(@owner_id)
	
		select @DiagId = diagram_id, @UIDFound = principal_id from dbo.sysdiagrams where principal_id = @owner_id and name = @diagramname 
		if(@DiagId IS NULL or (@IsDbo = 0 and @UIDFound <> @theId))
		begin
			RAISERROR ('Diagram does not exist or you do not have permission.', 16, 1)
			return -3
		end
	
		-- if((@u_name is not null) and (@new_diagramname = @diagramname))	-- nothing will change
		--	return 0;
	
		if(@u_name is null)
			select @DiagIdTarg = diagram_id from dbo.sysdiagrams where principal_id = @theId and name = @new_diagramname
		else
			select @DiagIdTarg = diagram_id from dbo.sysdiagrams where principal_id = @owner_id and name = @new_diagramname
	
		if((@DiagIdTarg is not null) and  @DiagId <> @DiagIdTarg)
		begin
			RAISERROR ('The name is already used.', 16, 1);
			return -2
		end		
	
		if(@u_name is null)
			update dbo.sysdiagrams set [name] = @new_diagramname, principal_id = @theId where diagram_id = @DiagId
		else
			update dbo.sysdiagrams set [name] = @new_diagramname where diagram_id = @DiagId
		return 0
	END
	
GO
/****** Object:  StoredProcedure [dbo].[sp_upgraddiagrams]    Script Date: 6/26/2019 11:14:26 AM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO

	CREATE PROCEDURE [dbo].[sp_upgraddiagrams]
	AS
	BEGIN
		IF OBJECT_ID(N'dbo.sysdiagrams') IS NOT NULL
			return 0;
	
		CREATE TABLE dbo.sysdiagrams
		(
			name sysname NOT NULL,
			principal_id int NOT NULL,	-- we may change it to varbinary(85)
			diagram_id int PRIMARY KEY IDENTITY,
			version int,
	
			definition varbinary(max)
			CONSTRAINT UK_principal_name UNIQUE
			(
				principal_id,
				name
			)
		);


		/* Add this if we need to have some form of extended properties for diagrams */
		/*
		IF OBJECT_ID(N'dbo.sysdiagram_properties') IS NULL
		BEGIN
			CREATE TABLE dbo.sysdiagram_properties
			(
				diagram_id int,
				name sysname,
				value varbinary(max) NOT NULL
			)
		END
		*/

		IF OBJECT_ID(N'dbo.dtproperties') IS NOT NULL
		begin
			insert into dbo.sysdiagrams
			(
				[name],
				[principal_id],
				[version],
				[definition]
			)
			select	 
				convert(sysname, dgnm.[uvalue]),
				DATABASE_PRINCIPAL_ID(N'dbo'),			-- will change to the sid of sa
				0,							-- zero for old format, dgdef.[version],
				dgdef.[lvalue]
			from dbo.[dtproperties] dgnm
				inner join dbo.[dtproperties] dggd on dggd.[property] = 'DtgSchemaGUID' and dggd.[objectid] = dgnm.[objectid]	
				inner join dbo.[dtproperties] dgdef on dgdef.[property] = 'DtgSchemaDATA' and dgdef.[objectid] = dgnm.[objectid]
				
			where dgnm.[property] = 'DtgSchemaNAME' and dggd.[uvalue] like N'_EA3E6268-D998-11CE-9454-00AA00A3F36E_' 
			return 2;
		end
		return 1;
	END
	
GO
ALTER DATABASE [sqltoazure] SET  READ_WRITE 
GO


Configuration option 'external scripts enabled' changed from 0 to 1. Run the RECONFIGURE statement to install.

Total execution time: 00:00:00.1829170

<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/thinking.jpg?raw=true"><b>For Further Study</b></p>

<br>
<br>

- [Primary Documentation on Data Tier Applications](https://docs.microsoft.com/en-us/sql/relational-databases/data-tier-applications/data-tier-applications?view=sql-server-2017)


<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/education1.png?raw=true"><b>Next</b>: Query your Data</p>

Next, you'll query your solutions, components, references and more in *02-QueryProblemsAndSolutionsDB*. Open that Notebook to continue.
